In [1]:
#directory='C:\\Users\\Administrator\\Documents\\GitHub\\Code\\2019\\Udemy\\Computer Vision Using Open CV\\Master OpenCV\\images'
directory_home='C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Udemy\\Computer Vision Using Open CV\\Master OpenCV\\images'

import os
os.chdir(directory_home)

In [1]:
import cv2

## Mini Project # 4 - Finding Waldo

In [3]:
import cv2
import numpy as np

# Load input image and convert to grayscale
image = cv2.imread('WaldoBeach.jpg')
cv2.imshow('Where is Waldo?', image)
cv2.waitKey(0)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load Template image
template = cv2.imread('waldo.jpg',0)

result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF)

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
print(max_loc)

#Create Bounding Box
top_left = max_loc
bottom_right = (top_left[0] + 50, top_left[1] + 50)
cv2.rectangle(image, top_left, bottom_right, (0,0,255), 5)

cv2.imshow('Where is Waldo?', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(484, 262)


## Finding Corners

### Notes on Template Matching

There are a variety of methods to perform template matching, but in this case we are using the correlation coefficient which is specified by the flag **cv2.TM_CCOEFF.**

So what exactly is the cv2.matchTemplate function doing?
Essentially, this function takes a “sliding window” of our waldo query image and slides it across our puzzle image from left to right and top to bottom, one pixel at a time. Then, for each of these locations, we compute the correlation coefficient to determine how “good” or “bad” the match is. 

Regions with sufficiently high correlation can be considered “matches” for our waldo template.
From there, all we need is a call to cv2.minMaxLoc on Line 22 to find where our “good” matches are.
That’s really all there is to template matching!

http://docs.opencv.org/2.4/modules/imgproc/doc/object_detection.html      

In [10]:
import cv2
import numpy as np

# Load image then grayscale
image = cv2.imread('chess.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# The cornerHarris function requires the array datatype to be float32
gray = np.float32(gray)

harris_corners = cv2.cornerHarris(gray, 3, 3, 0.05)

#We use dilation of the corner points to enlarge them\
kernel = np.ones((7,7),np.uint8)
harris_corners = cv2.dilate(harris_corners, kernel, iterations = 2)
print(harris_corners)
print(harris_corners > 0.025 * harris_corners.max())
# Threshold for an optimal value, it may vary depending on the image.
image[harris_corners > 0.025 * harris_corners.max()] = [255, 127, 127]

cv2.imshow('Harris Corners', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[67.52585    67.52585    67.52585    ...  0.21813272  0.21813272
   0.21813272]
 [67.52585    67.52585    67.52585    ...  0.23221451  0.23221451
   0.23221451]
 [67.52585    67.52585    67.52585    ...  0.31414932  0.31414932
   0.31414932]
 ...
 [ 0.13541663  0.13541663  0.13541663 ...  0.31549966  0.31549966
   0.31549966]
 [ 0.13541663  0.13541663  0.13541663 ...  0.31549966  0.31549966
   0.31549966]
 [ 0.13541663  0.13541663  0.13541663 ...  0.31549966  0.31549966
   0.31549966]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


#### Harris Corner Detection is an algorithm developed in 1998 for corner detection  (http://www.bmva.org/bmvc/1988/avc-88-023.pdf) and works fairly well.

**cv2.cornerHarris**(input image, block size, ksize, k)
- Input image - should be grayscale and float32 type.
- blockSize - the size of neighborhood considered for corner detection
- ksize - aperture parameter of Sobel derivative used.
- k - harris detector free parameter in the equation
- **Output** – array of corner locations (x,y)




## Improved Corner Detection using - Good Features to Track

In [12]:
import cv2
import numpy as np

img = cv2.imread('chess.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# We specific the top 50 corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 15)

for corner in corners:
    x, y = corner[0]
    x = int(x)
    y = int(y)
    cv2.rectangle(img,(x-10,y-10),(x+10,y+10),(0,255,0), 2)
    
cv2.imshow("Corners Found", img)
cv2.waitKey()
cv2.destroyAllWindows()

**cv2.goodFeaturesToTrack**(input image, maxCorners, qualityLevel, minDistance)

- Input Image - 8-bit or floating-point 32-bit, single-channel image.
- maxCorners – Maximum number of corners to return. If there are more corners than are found, the strongest of them is returned.
- qualityLevel – Parameter characterizing the minimal accepted quality of image corners. The parameter value is multiplied by the best corner quality measure (smallest eigenvalue). The corners with the quality measure less than the product are rejected. For example, if the best corner has the quality measure = 1500, and the  qualityLevel=0.01 , then all the corners with the quality - - measure less than 15 are rejected.
- minDistance – Minimum possible Euclidean distance between the returned corners.


In [18]:
import cv2
import numpy as np

image = cv2.imread('input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Create SIFT Feature Detector object
sift = cv2.xfeatures2d.SIFT_create()

#Detect key points

keypoints = sift.detect(gray, None)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich key points on input image
blank = np.zeros((1,1)) 
image = cv2.drawKeypoints(image, keypoints, blank, (0,255,255),flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('Feature Method - SIFT', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of keypoints Detected:  1893


In [15]:
import cv2
sift = cv2.xfeatures2d.SIFT_create()